In [7]:
import os 
import json 
import itertools
from tqdm import tqdm 
fine_file = '/home/haoquan/GAIIC2022/temp/tmp_data/lhq_data/divided/title/order/fine40000.txt' 

# get attr_dict
DATA_DIR = '/home/haoquan/GAIIC2022/original_data/train'
attr_dict_file = os.path.join(DATA_DIR, 'attr_to_attrvals.json')
def load_attr_dict(file):
    # 读取属性字典
    with open(file, 'r') as f:
        attr_dict = {}
        for attr, attrval_list in json.load(f).items():
            attrval_list = list(map(lambda x: x.split('='), attrval_list))
            attr_dict[attr] = list(itertools.chain.from_iterable(attrval_list))
    return attr_dict
attr_dict = load_attr_dict(attr_dict_file)
            
# 特殊的几个属性替换
for query, attrs in attr_dict.items():
    attrs = attrs.copy()
    for i, attr in enumerate(attrs):
        if query=='裤门襟' and attr=='拉链':
            attr_dict[query][i] = '拉链裤'
        if query=='裤门襟' and attr=='系带':
            attr_dict[query][i] = '系带裤'
        if query=='裤门襟' and attr=='松紧':
            attr_dict[query][i] = '松紧裤'
        if query=='闭合方式' and attr=='拉链':
            attr_dict[query][i] = '拉链鞋'
        if query=='闭合方式' and attr=='系带':
            attr_dict[query][i] = '系带鞋'

# 对于搜索属性可能出错的部分，重调了顺序
for query, attr_list in attr_dict.items():
    if query == '领型':
        attr_list.remove('高领')
        attr_list.append('高领')
    if query == '衣长':
        attr_list.remove('短款')
        attr_list.append('短款')
    if query == '衣长':
        attr_list.remove('长款')
        attr_list.append('长款')
    if query == '裙长':
        attr_list.remove('短裙')
        attr_list.append('短裙')
    if query == '裙长':
        attr_list.remove('长裙')
        attr_list.append('长裙')

In [8]:
save_file = '/home/haoquan/GAIIC2022/temp/tmp_data/lhq_data/divided/title/order/fine40000_test.txt'
rets = []
with open(fine_file, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        title = data['title']
        key_attr = {}
        match = {}
        match['图文'] = 1
        # 属性提取
        for query, attr_list in attr_dict.items():
            for attr in attr_list:
                if attr in title:
                    key_attr[query] = attr
                    match[query] = 1 
                    break 
        data['key_attr'] = key_attr
        data['match'] = match

        rets.append(json.dumps(data, ensure_ascii=False)+'\n')
        if len(rets) > 200:
            break 
        
print(len(rets))
with open(save_file, 'w') as f:
    f.writelines(rets)

200it [00:00, 854.82it/s]

201
